In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from citipy import citipy
import json
import time; time.time()

# Range of latitudes and longitudes
lat_range = (64, 20)
lng_range = (-154, -65)

# Output File (CSV)
output_data_file = "data/cities.csv"

# Import API key
from api_keys import weather_key

In [203]:
weather_data_df = pd.read_csv("data/tornadoes.csv")
weather_data_df.head()

,EVENT_ID,CZ_NAME_STR,BEGIN_LOCATION,BEGIN_DATE,BEGIN_TIME,EVENT_TYPE,MAGNITUDE,TOR_F_SCALE,DEATHS_DIRECT,INJURIES_DIRECT,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EVENT_NARRATIVE,EPISODE_NARRATIVE,ABSOLUTE_ROWNUMBER
0,866322,CARTERET CO.,TRUTTNEYS LNDG,11/1/2019,15,Tornado,,EF0,0,0,...,1,NNW,TRUTTNEYS LNDG,34.7394,-77.1094,34.7411,-77.1053,A brief tornado occurred near the intersection...,A strong cold front moved across Eastern North...,1
1,859389,MARTIN CO.,WAVELAND,11/5/2019,1620,Tornado,,EF0,0,0,...,2,ENE,WAVELAND,27.2140,-80.1729,27.2123,-80.1766,Martin County Beach Patrol reported a waterspo...,A strong thunderstorm produced a waterspout ju...,2
2,864077,MOBILE CO.,PRICHARD,11/7/2019,1335,Tornado,,EF0,0,0,...,0,SSE,PRICHARD,30.7248,-88.0548,30.7260,-88.0486,The storm survey team surveyed the areas aroun...,Thunderstorms moved across the area during the...,3
3,857862,FRANKLIN PAR.,BASKIN,11/26/2019,1803,Tornado,,EF2,1,1,...,4,NE,BASKIN,32.2351,-91.7783,32.3184,-91.7095,This tornado began on Roger Miller Road as it ...,"Warm, moist air was in place across northeast ...",4
4,857944,CLAIBORNE PAR.,MAHAN,11/26/2019,1820,Tornado,,EF1,0,0,...,2,NNE,MAHAN,32.9017,-93.0423,32.9076,-93.0343,An EF-1 tornado with estimated maximum winds a...,A deep southwesterly flow aloft ahead of a lar...,5


In [204]:
tornado_df = weather_data_df.loc[:,["BEGIN_LOCATION", "STATE_ABBR", "BEGIN_DATE", "EVENT_TYPE", "TOR_F_SCALE", "DEATHS_DIRECT", "BEGIN_LAT", "BEGIN_LON", "END_LOCATION"]]
tornado_df.head()

,BEGIN_LOCATION,STATE_ABBR,BEGIN_DATE,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,BEGIN_LAT,BEGIN_LON,END_LOCATION
0,TRUTTNEYS LNDG,NC,11/1/2019,Tornado,EF0,0,34.7394,-77.1094,TRUTTNEYS LNDG
1,WAVELAND,FL,11/5/2019,Tornado,EF0,0,27.2140,-80.1729,WAVELAND
2,PRICHARD,AL,11/7/2019,Tornado,EF0,0,30.7248,-88.0548,PRICHARD
3,BASKIN,LA,11/26/2019,Tornado,EF2,1,32.2351,-91.7783,BASKIN
4,MAHAN,LA,11/26/2019,Tornado,EF1,0,32.9017,-93.0423,MAHAN


In [205]:
gmaps.configure(api_key = weather_key)

In [206]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

538

In [207]:
for index, row in tornado_df.iterrows():
    base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=04c41275fea3e8291170fd88095dde0f"
    beg_lat= row['BEGIN_LAT']
    beg_lon= row['BEGIN_LON']

In [208]:
tornado_df = tornado_df.rename(columns={"BEGIN_LOCATION": "City", "BEGIN_LAT":"Lat", "BEGIN_LON":"Lng", "BEGIN_DATE": "Date"})
tornado_df.head()

,City,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION
0,TRUTTNEYS LNDG,NC,11/1/2019,Tornado,EF0,0,34.7394,-77.1094,TRUTTNEYS LNDG
1,WAVELAND,FL,11/5/2019,Tornado,EF0,0,27.2140,-80.1729,WAVELAND
2,PRICHARD,AL,11/7/2019,Tornado,EF0,0,30.7248,-88.0548,PRICHARD
3,BASKIN,LA,11/26/2019,Tornado,EF2,1,32.2351,-91.7783,BASKIN
4,MAHAN,LA,11/26/2019,Tornado,EF1,0,32.9017,-93.0423,MAHAN


In [209]:
cities_lat_lng = requests.get(base_url)

In [210]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=04c41275fea3e8291170fd88095dde0f"
# List of city data
city_data = []
# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")
# Create counters
record_count = 1
set_count = 1
# Loop through all the cities in our list
for x, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes
    if (x % 50 == 0 and x >= 50):
        set_count += 1
        record_count = 0
    # Create endpoint URL with each city
    city_url = url + "&q=" + city
    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))
    # Add 1 to the record count
    record_count += 1
    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        # Parse out the max temp, humidity, and cloudiness
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_date = city_weather["dt"]
        # Append the City information into city_data list
        city_data.append({"City": city, 
                          "Lat": city_lat, 
                          "Lng": city_lng, 
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tucumcari
Processing Record 2 of Set 1 | la ronge
Processing Record 3 of Set 1 | selma
Processing Record 4 of Set 1 | banes
Processing Record 5 of Set 1 | pendleton
Processing Record 6 of Set 1 | haines junction
Processing Record 7 of Set 1 | madison
Processing Record 8 of Set 1 | edson
Processing Record 9 of Set 1 | meadow lake
Processing Record 10 of Set 1 | guerrero negro
Processing Record 11 of Set 1 | middlebury
Processing Record 12 of Set 1 | kodiak
Processing Record 13 of Set 1 | cockburn town
Processing Record 14 of Set 1 | prince george
Processing Record 15 of Set 1 | north bend
Processing Record 16 of Set 1 | san quintin
Processing Record 17 of Set 1 | conroe
Processing Record 18 of Set 1 | fort nelson
Processing Record 19 of Set 1 | hilo
Processing Record 20 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 21 of Set 1 | half moon bay
Processing Record 22 of Set 1 |

Processing Record 43 of Set 4 | great falls
Processing Record 44 of Set 4 | campechuela
Processing Record 45 of Set 4 | show low
Processing Record 46 of Set 4 | rockport
Processing Record 47 of Set 4 | enid
Processing Record 48 of Set 4 | high level
Processing Record 49 of Set 4 | cedar city
Processing Record 0 of Set 5 | moberly
Processing Record 1 of Set 5 | abilene
Processing Record 2 of Set 5 | brigantine
Processing Record 3 of Set 5 | valentin gomez farias
Processing Record 4 of Set 5 | bucerias
Processing Record 5 of Set 5 | russellville
Processing Record 6 of Set 5 | pittsfield
Processing Record 7 of Set 5 | sheridan
Processing Record 8 of Set 5 | seguin
Processing Record 9 of Set 5 | canon city
Processing Record 10 of Set 5 | ville-marie
Processing Record 11 of Set 5 | prince rupert
Processing Record 12 of Set 5 | batavia
Processing Record 13 of Set 5 | el sauzal
Processing Record 14 of Set 5 | dzilam gonzalez
Processing Record 15 of Set 5 | aberdeen
Processing Record 16 of Set

Processing Record 37 of Set 8 | ridgecrest
Processing Record 38 of Set 8 | ozark
Processing Record 39 of Set 8 | black forest
Processing Record 40 of Set 8 | cape canaveral
Processing Record 41 of Set 8 | toppenish
Processing Record 42 of Set 8 | georgetown
Processing Record 43 of Set 8 | jasper
Processing Record 44 of Set 8 | fort smith
Processing Record 45 of Set 8 | crossfield
Processing Record 46 of Set 8 | el centro
Processing Record 47 of Set 8 | mitchell
Processing Record 48 of Set 8 | vega de alatorre
Processing Record 49 of Set 8 | high prairie
Processing Record 0 of Set 9 | tierranueva
Processing Record 1 of Set 9 | altoona
Processing Record 2 of Set 9 | bathurst
Processing Record 3 of Set 9 | bismarck
Processing Record 4 of Set 9 | miramar
Processing Record 5 of Set 9 | muskegon
Processing Record 6 of Set 9 | rancho cucamonga
Processing Record 7 of Set 9 | parker
Processing Record 8 of Set 9 | bainbridge
Processing Record 9 of Set 9 | whitefish
Processing Record 10 of Set 9 

In [211]:
# Export the City_Data into a csv
city_data_pd.to_csv(output_data_file, index_label="City")

In [212]:
city_data_pd = pd.DataFrame(city_data)
city_data_pd.count()


City          528
Lat           528
Lng           528
Max Temp      528
Humidity      528
Cloudiness    528
Wind Speed    528
Country       528
Date          528
dtype: int64

In [213]:
city_data_pd.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tucumcari,35.1717,-103.7250,48.20,93,75,16.11,US,1615664143
1,la ronge,55.1001,-105.2842,30.20,50,2,8.05,CA,1615664144
2,selma,36.5708,-119.6121,63.00,47,1,3.44,US,1615664101
3,banes,20.9625,-75.7186,84.20,54,40,16.11,CU,1615664144
4,pendleton,45.6721,-118.7886,55.99,43,1,6.91,US,1615663934


In [214]:
america_city = city_data_pd[(city_data_pd["Country"]=="US")]
america_city

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tucumcari,35.1717,-103.7250,48.20,93,75,16.11,US,1615664143
2,selma,36.5708,-119.6121,63.00,47,1,3.44,US,1615664101
4,pendleton,45.6721,-118.7886,55.99,43,1,6.91,US,1615663934
6,madison,34.7334,-86.5667,75.00,64,90,5.28,US,1615664145
8,meadow lake,34.8014,-106.5436,48.00,31,75,16.11,US,1615664146
...,...,...,...,...,...,...,...,...,...
517,kirksville,40.1948,-92.5833,57.99,41,1,14.97,US,1615664185
519,ann arbor,42.2776,-83.7409,48.99,23,1,8.05,US,1615664156
522,north platte,41.1239,-100.7654,39.20,87,90,12.66,US,1615664132
523,buckeye,33.3703,-112.5838,60.01,48,94,3.00,US,1615664286


In [215]:
tornado_df2=tornado_df.copy()

In [216]:
tornado_df["City"] = tornado_df["City"].str.lower()
tornado_df

,City,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION
0,truttneys lndg,NC,11/1/2019,Tornado,EF0,0,34.7394,-77.1094,TRUTTNEYS LNDG
1,waveland,FL,11/5/2019,Tornado,EF0,0,27.2140,-80.1729,WAVELAND
2,prichard,AL,11/7/2019,Tornado,EF0,0,30.7248,-88.0548,PRICHARD
3,baskin,LA,11/26/2019,Tornado,EF2,1,32.2351,-91.7783,BASKIN
4,mahan,LA,11/26/2019,Tornado,EF1,0,32.9017,-93.0423,MAHAN
...,...,...,...,...,...,...,...,...,...
495,lamont,GA,4/13/2020,Tornado,EF0,0,32.9896,-84.1225,LAMONT
496,red oak,GA,4/13/2020,Tornado,EF0,0,33.6288,-84.5263,RED OAK
497,collier,GA,4/13/2020,Tornado,EF3,0,33.0298,-84.0330,BRIGHTON MILLS
498,roberts xrds,GA,4/13/2020,Tornado,EF0,0,33.4560,-83.6449,ROBERTS XRDS


In [217]:
america_city.City = america_city.City.astype(str)
tornado_df.City = tornado_df.City.astype(str)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [228]:
concat_df=pd.concat([tornado_df.set_index("City"), america_city.set_index("City")], join="outer")
concat_df


,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION,Max Temp,Humidity,Cloudiness,Wind Speed,Country
City,,,,,,,,,,,,,
truttneys lndg,NC,11/1/2019,Tornado,EF0,0.0,34.7394,-77.1094,TRUTTNEYS LNDG,NaN,NaN,NaN,NaN,NaN
waveland,FL,11/5/2019,Tornado,EF0,0.0,27.2140,-80.1729,WAVELAND,NaN,NaN,NaN,NaN,NaN
prichard,AL,11/7/2019,Tornado,EF0,0.0,30.7248,-88.0548,PRICHARD,NaN,NaN,NaN,NaN,NaN
baskin,LA,11/26/2019,Tornado,EF2,1.0,32.2351,-91.7783,BASKIN,NaN,NaN,NaN,NaN,NaN
mahan,LA,11/26/2019,Tornado,EF1,0.0,32.9017,-93.0423,MAHAN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
kirksville,NaN,1615664185,NaN,NaN,NaN,40.1948,-92.5833,NaN,57.99,41.0,1.0,14.97,US
ann arbor,NaN,1615664156,NaN,NaN,NaN,42.2776,-83.7409,NaN,48.99,23.0,1.0,8.05,US
north platte,NaN,1615664132,NaN,NaN,NaN,41.1239,-100.7654,NaN,39.20,87.0,90.0,12.66,US


In [233]:
concat_df["Date"]=(pd.to_datetime(concat_df["Date"]))
concat_df

,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION,Max Temp,Humidity,Cloudiness,Wind Speed,Country
City,,,,,,,,,,,,,
truttneys lndg,NC,2019-01-11,Tornado,EF0,0.0,34.7394,-77.1094,TRUTTNEYS LNDG,NaN,NaN,NaN,NaN,NaN
waveland,FL,2019-05-11,Tornado,EF0,0.0,27.2140,-80.1729,WAVELAND,NaN,NaN,NaN,NaN,NaN
prichard,AL,2019-07-11,Tornado,EF0,0.0,30.7248,-88.0548,PRICHARD,NaN,NaN,NaN,NaN,NaN
baskin,LA,2019-11-26,Tornado,EF2,1.0,32.2351,-91.7783,BASKIN,NaN,NaN,NaN,NaN,NaN
mahan,LA,2019-11-26,Tornado,EF1,0.0,32.9017,-93.0423,MAHAN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
kirksville,NaN,1970-01-01,NaN,NaN,NaN,40.1948,-92.5833,NaN,57.99,41.0,1.0,14.97,US
ann arbor,NaN,1970-01-01,NaN,NaN,NaN,42.2776,-83.7409,NaN,48.99,23.0,1.0,8.05,US
north platte,NaN,1970-01-01,NaN,NaN,NaN,41.1239,-100.7654,NaN,39.20,87.0,90.0,12.66,US


In [234]:
concat_df["Date"] = concat_df["Date"].map(lambda ts: ts.strftime("%m-%d-%Y"))
concat_df

,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION,Max Temp,Humidity,Cloudiness,Wind Speed,Country
City,,,,,,,,,,,,,
truttneys lndg,NC,01-11-2019,Tornado,EF0,0.0,34.7394,-77.1094,TRUTTNEYS LNDG,NaN,NaN,NaN,NaN,NaN
waveland,FL,05-11-2019,Tornado,EF0,0.0,27.2140,-80.1729,WAVELAND,NaN,NaN,NaN,NaN,NaN
prichard,AL,07-11-2019,Tornado,EF0,0.0,30.7248,-88.0548,PRICHARD,NaN,NaN,NaN,NaN,NaN
baskin,LA,11-26-2019,Tornado,EF2,1.0,32.2351,-91.7783,BASKIN,NaN,NaN,NaN,NaN,NaN
mahan,LA,11-26-2019,Tornado,EF1,0.0,32.9017,-93.0423,MAHAN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
kirksville,NaN,01-01-1970,NaN,NaN,NaN,40.1948,-92.5833,NaN,57.99,41.0,1.0,14.97,US
ann arbor,NaN,01-01-1970,NaN,NaN,NaN,42.2776,-83.7409,NaN,48.99,23.0,1.0,8.05,US
north platte,NaN,01-01-1970,NaN,NaN,NaN,41.1239,-100.7654,NaN,39.20,87.0,90.0,12.66,US


In [236]:
filteredtime_df = concat_df.loc[(concat_df["Date"] >= "11-01-2019")]
filteredtime_df

,STATE_ABBR,Date,EVENT_TYPE,TOR_F_SCALE,DEATHS_DIRECT,Lat,Lng,END_LOCATION,Max Temp,Humidity,Cloudiness,Wind Speed,Country
City,,,,,,,,,,,,,
baskin,LA,11-26-2019,Tornado,EF2,1.0,32.2351,-91.7783,BASKIN,NaN,NaN,NaN,NaN,NaN
mahan,LA,11-26-2019,Tornado,EF1,0.0,32.9017,-93.0423,MAHAN,NaN,NaN,NaN,NaN,NaN
tendal,LA,11-26-2019,Tornado,EFU,0.0,32.4565,-91.3847,WAVERLY,NaN,NaN,NaN,NaN,NaN
whites,MS,11-26-2019,Tornado,EF2,0.0,32.0586,-90.1282,ROOK HILL,NaN,NaN,NaN,NaN,NaN
johns,MS,11-27-2019,Tornado,EF1,0.0,32.1338,-89.8041,RUFUS,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
east cleveland,TN,12-04-2020,Tornado,EF1,0.0,35.1663,-84.8242,CLIMER,NaN,NaN,NaN,NaN,NaN
julian chapel,TN,12-04-2020,Tornado,EF1,0.0,35.2046,-84.7751,HAUN MILL,NaN,NaN,NaN,NaN,NaN
benton station,TN,12-04-2020,Tornado,EF0,0.0,35.2132,-84.7013,BENTON STATION,NaN,NaN,NaN,NaN,NaN
